In [10]:
import numpy as np
import tensorflow as tf

#### Fetch the openml digit recognition data

In [14]:
from sklearn.datasets import fetch_openml
data = fetch_openml('mnist_784', as_frame=False)
X, y = data.data, data.target

In [15]:
X.shape

(70000, 784)

In [16]:
y.shape

(70000,)

#### Splitting the data into x_train, y_train, x_test, y_test:

In [17]:
x_train = X[:60000]
y_train = y[:60000]

x_test = X[60000:]
y_test = y[60000:]

##### Converting the datset x_train and x_test dataset into 3 channels

In [ ]:
x_train = x_train.astype(np.uint8)
x_test = x_test.astype(np.uint8)

x_train = np.repeat(x_train[:, :, :, np.newaxis], 3, axis=3)
x_test = np.repeat(x_test[:, :, :, np.newaxis], 3, axis=3)

##### Adding zero padding to the training dataset

In [2]:
# Creating the function to add zero padding to the training dataset

def zero_pad(X, pad):
    # Adding the zero padding of size pad to the height and width of X

    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode = 'constant', constant_values = (0, 0))
    # returning X_pad
    return X_pad

#### Single step convolution

Here we are creating single step convolution operation where kernel window is applied on the previous input image and multiply and added the all the pixel values from the input image. It will result into the output which is reduced version of the input image.

In [2]:
def Conv_single_step(a_prev_slice, W, b):
    #apply window W over a_prev_slice
    s = a_prev_slice * W
    Z = np.sum(s) + float(b)
    Z = np.float64(Z)

    return Z

#### Forward propagation in convolution neural network